# How to Read in Multiple Volume Scans Into a Single Object

## Imports

In [ ]:
import xradar as xd
import xarray as xr
import act
import glob
import pyart
from pathlib import Path

## Access the Data

In [ ]:
files = sorted(glob.glob("sample-subset-data/*"))
files

In [ ]:
ds = xr.open_dataset(files[0], engine="cfradial1", group="sweep_0")

In [ ]:
def fix_angle(ds):
    ds["time"] = ds.time.load()  # Convert time from dask to numpy

    start_ang = 0  # Set consistent start/end values
    stop_ang = 360
    angle_dict = xd.util.extract_angle_parameters(ds)

    angle_res = 0.3
    direction = angle_dict["direction"]

    # first find exact duplicates and remove
    ds = xd.util.remove_duplicate_rays(ds)

    # second reindex according to retrieved parameters
    ds = xd.util.reindex_angle(
        ds, start_ang, stop_ang, angle_res, direction, method="nearest"
    )

    ds = ds.expand_dims("volume_time")  # Expand for volumes for concatenation

    ds["volume_time"] = [ds.time.min("azimuth").values]

    return ds

In [ ]:
# Concatenate in xarray ds
ds = xr.open_mfdataset(
    files,
    preprocess=fix_angle,
    engine="cfradial1",
    group="sweep_0",
    concat_dim="volume_time",
    combine="nested",
)